In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
NB_USERS = 5000
FEAT_USER = 3
NB_ITEMS = 100
FEAT_ITEM = 6

NB_EPOCHS = 500
LAMBDA_REG = 1e-5
# LAMBDA_REG = 0
# learning_rate = 0.001

In [3]:
# Load data
users = pd.read_csv('data/sushi/sushi3.udata', sep='\t', names=('uid', 'gender', 'age', 'time', 'old_prefecture', 'old_region', 'old_eastwest', 'prefecture', 'region', 'eastwest', 'same'))
items = pd.read_csv('data/sushi/sushi3.idata', sep='\t', names=('iid', 'name', 'style', 'major', 'minor', 'heaviness', 'frequency', 'price', 'popularity'))
R = pd.read_csv('data/sushi/sushi3b.5000.10.score', sep=' ', header=None)
triplets = []
for i, line in enumerate(np.array(R)):
    for j, v in enumerate(line):
        if v != -1:
            triplets.append((i, j, v))
df_ratings = pd.DataFrame(triplets, columns=('user', 'item', 'rating'))
train, test = train_test_split(df_ratings, test_size=0.2, shuffle=True)

In [4]:
# TF
A = tf.constant(np.array(users[['age', 'gender', 'region']]).astype(np.float32))
B = tf.constant(np.array(items[['heaviness', 'frequency', 'price', 'popularity', 'style', 'major']]).astype(np.float32))

W_V = tf.get_variable('W_V', shape=[NB_ITEMS, FEAT_USER], dtype=np.float32, initializer=tf.truncated_normal_initializer(stddev=1))
W_U = tf.get_variable('W_U', shape=[NB_USERS, FEAT_ITEM], dtype=np.float32, initializer=tf.truncated_normal_initializer(stddev=1))
M = tf.get_variable('M', shape=[FEAT_USER, FEAT_ITEM], dtype=np.float32, initializer=tf.truncated_normal_initializer(stddev=1))
user_bias = tf.get_variable("user_bias", shape=[NB_USERS],
                            initializer=tf.truncated_normal_initializer(stddev=1))
item_bias = tf.get_variable("item_bias", shape=[NB_ITEMS],
                            initializer=tf.truncated_normal_initializer(stddev=1))

rat_avg =  tf.get_variable("rat_avg", shape=[1],
                            initializer=tf.truncated_normal_initializer(stddev=1))

user_batch = tf.placeholder(tf.int32, shape=[None])
item_batch = tf.placeholder(tf.int32, shape=[None])
rate_batch = tf.placeholder(tf.float32, shape=[None])

weight_items = tf.nn.embedding_lookup(W_V, item_batch)
weight_users = tf.nn.embedding_lookup(W_U, user_batch)

bias_items = tf.nn.embedding_lookup(item_bias, item_batch)
bias_users = tf.nn.embedding_lookup(user_bias, user_batch)

feat_items = tf.nn.embedding_lookup(B, item_batch)
feat_users = tf.nn.embedding_lookup(A, user_batch)

pred = (tf.reduce_sum(tf.multiply(feat_users, weight_items), 1)
        + tf.reduce_sum(tf.multiply(feat_items, weight_users), 1)
        + bias_items
        + bias_users)

#pred = (tf.reduce_sum(tf.multiply(weight_users, weight_items), 1)
#        + bias_items
#        + bias_users)


#pred = ( bias_items + bias_users)
#pred = rat_avg

# pred = (tf.reduce_sum(tf.multiply(tf.matmul(feat_users, M), feat_items), 1)
#         + bias_items
#         + bias_users)
cost_l2 = tf.losses.mean_squared_error(rate_batch, pred)

l2_user = tf.nn.l2_loss(weight_users)
l2_item = tf.nn.l2_loss(weight_items)
l2_bias_user = tf.nn.l2_loss(bias_users)
l2_bias_item = tf.nn.l2_loss(bias_items)
regularizer = tf.add(l2_user, l2_item)
regularizer = tf.add(regularizer, l2_bias_user)
regularizer = tf.add(regularizer, l2_bias_item)
# regularizer = tf.nn.l2_loss(M)
penalty = tf.constant(LAMBDA_REG, dtype=tf.float32, shape=[])
cost = tf.add(cost_l2, tf.multiply(regularizer, penalty))

In [5]:
global_step = tf.train.get_global_step()
train_op = tf.train.AdamOptimizer(0.1).minimize(cost, global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(NB_EPOCHS):
        _, train_pred, train_mse, reg, pen, train_cost = sess.run([train_op, pred, cost_l2, regularizer, penalty, cost], feed_dict={
            user_batch: train['user'],
            item_batch: train['item'],
            rate_batch: train['rating']
        })
        test_pred, test_mse = sess.run([pred, cost_l2], feed_dict={
            user_batch: test['user'],
            item_batch: test['item'],
            rate_batch: test['rating']
        })
        print('train rmse', train_mse ** 0.5, 'test rmse', test_mse ** 0.5)
        # print('reg', reg, 'full cost', train_cost)

train rmse 6.613511110244109 test rmse 5.8129410114953854
train rmse 5.690195974719402 test rmse 5.08279457781001
train rmse 4.868665835004437 test rmse 4.453771283283284
train rmse 4.158459069154581 test rmse 3.92954795392452
train rmse 3.5683466801994355 test rmse 3.5106454806743166
train rmse 3.1028519647650903 test rmse 3.1921659240142874
train rmse 2.7595864444995244 test rmse 2.96185803761307
train rmse 2.5258527127937005 test rmse 2.802130763752682
train rmse 2.379704183418568 test rmse 2.693699716333222
train rmse 2.294761879396179 test rmse 2.6186983672448494
train rmse 2.246025814189991 test rmse 2.5627771553325287
train rmse 2.2138920645037565 test rmse 2.5155079500440105
train rmse 2.1849936159158494 test rmse 2.4698334260132193
train rmse 2.1512500455724433 test rmse 2.421666979316049
train rmse 2.1089630855044637 test rmse 2.369416147738626
train rmse 2.057671646244795 test rmse 2.313120990575507
train rmse 1.9986586244844196 test rmse 2.2537298017750884
train rmse 1.9339

train rmse 0.7805141034852867 test rmse 1.2123462972848993
train rmse 0.7804208551780888 test rmse 1.2122830206617403
train rmse 0.7803317204599552 test rmse 1.2122247560484136
train rmse 0.7802440270143626 test rmse 1.212169832325573
train rmse 0.780160067346678 test rmse 1.2121166272058617
train rmse 0.7800793460078036 test rmse 1.2120659769091269
train rmse 0.7799999153919663 test rmse 1.2120154720300842
train rmse 0.7799227311913843 test rmse 1.2119656535686119
train rmse 0.7798482526596832 test rmse 1.2119149477799447
train rmse 0.7797756015333149 test rmse 1.2118634037369957
train rmse 0.7797041667617006 test rmse 1.211811267264871
train rmse 0.7796356306233545 test rmse 1.2117589809839662
train rmse 0.7795696115908755 test rmse 1.211707381115821
train rmse 0.7795047339392909 test rmse 1.211657894335391
train rmse 0.7794421450180041 test rmse 1.2116110620416194
train rmse 0.7793813865173579 test rmse 1.2115670813276345
train rmse 0.7793223823828177 test rmse 1.2115260508999703
tr

train rmse 0.7772772295683035 test rmse 1.2106580350245357
train rmse 0.7772736254169238 test rmse 1.210612099519214
train rmse 0.777272973600528 test rmse 1.2106499115089773
train rmse 0.7772700595911335 test rmse 1.2106367168342016
train rmse 0.7772687176094488 test rmse 1.210623374311609
train rmse 0.777267605680012 test rmse 1.2106550317918015
train rmse 0.7772651134185994 test rmse 1.210625934511847
train rmse 0.7772646916504928 test rmse 1.2106369630048235
train rmse 0.777262889547823 test rmse 1.2106496161073843
train rmse 0.7772614325255883 test rmse 1.2106246544124049
train rmse 0.7772608190417251 test rmse 1.2106454312437416
train rmse 0.7772579816725589 test rmse 1.2106432157218392
train rmse 0.7772569847566092 test rmse 1.210628445472206
train rmse 0.7772561412113453 test rmse 1.210649419172949
train rmse 0.7772545691472743 test rmse 1.2106378984527306
train rmse 0.7772542240596135 test rmse 1.2106335166115607
train rmse 0.7772534188544761 test rmse 1.2106499115089773
train

train rmse 0.777209092941926 test rmse 1.2106330735031436
train rmse 0.7772082876900317 test rmse 1.2106369630048235
train rmse 0.7772092846684445 test rmse 1.2106351413410354
train rmse 0.777208939560677 test rmse 1.210634501296351
train rmse 0.777209169632539 test rmse 1.2106349444042457
train rmse 0.7772088245247206 test rmse 1.2106342551252287
train rmse 0.7772085561074225 test rmse 1.2106340581882946
train rmse 0.7772084410714092 test rmse 1.2106338612513285
train rmse 0.7772082493446826 test rmse 1.2106346489990008
train rmse 0.777209553085491 test rmse 1.2106322365201356
train rmse 0.7772078658910874 test rmse 1.2106357321512131
train rmse 0.7772099365382538 test rmse 1.21063179341125
train rmse 0.7772081343086239 test rmse 1.21063563368287
train rmse 0.7772112019310286 test rmse 1.2106313010678538
train rmse 0.777209169632539 test rmse 1.2106360275561938
train rmse 0.7772115470377914 test rmse 1.210630661021139
train rmse 0.7772074057465235 test rmse 1.2106338612513285
train rm